In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision
from PIL import Image
import seaborn as sns
import os

In [2]:
num_classes = 2
my_model = torch.hub.load('pytorch/vision', 'resnet34')
my_model.fc = nn.Sequential(nn.Linear(my_model.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))
my_model.load_state_dict(torch.load('./model.pth'))
my_model.eval().to('cuda')

Using cache found in /home/breno-cavalcanti/.cache/torch/hub/pytorch_vision_master


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
img_dimensions = 224
batch_size = 32
num_workers = 6

test_data_path = "./test2"
img_test_transforms = transforms.Compose([
    transforms.Resize((img_dimensions,img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_test_transforms)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)



def test_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_data_loader:
            images, labels = data[0].to('cuda'), data[1].to('cuda')
            outputs = model(images)
            _ , predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'correct: {correct}  total: {total}')
    print(f'accuracy = {round((correct / total * 100), 2)}%')
test_model(my_model)

correct: 705  total: 714
accuracy = 98.74%


In [4]:
import os
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

def make_prediction(model, filename):
    labels, _ = find_classes('./train')
    img = Image.open(filename)
    img = img_test_transforms(img)
    img = img.unsqueeze(0)
    prediction = model(img.to(device))
    prediction = prediction.argmax()
    print(prediction)

In [ ]:
path = 
for i in range(1, 12):
    print(f'{i}', end= ' ')
    make_prediction(my_model, f'./{path}/{i}.jpg')
